In [ ]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPStreamableHttpPlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,AgentResponseItem
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion


from semantic_kernel.contents import ChatMessageContent
from dotenv import load_dotenv
from os import environ
from tracing import set_up_all
from history_store import CosmosChatHistoryStore, ChatRole
from evaluation import Evaluation
import json

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
import uuid

load_dotenv(override=True)


session_id = str(uuid.uuid4())


In [ ]:

history_store = CosmosChatHistoryStore()
history = await history_store.load(session_id)


## Introduction to Evaluation with the Azure AI SDK

Evaluation is a critical part of building reliable and trustworthy generative AI applications. It ensures that AI outputs are grounded, coherent, and aligned with the intended context, helping to prevent issues like fabrication, irrelevance, and harmful content.

The **Azure AI Evaluation SDK** allows you to systematically evaluate the performance of your AI workflows directly in your development environment. This helps build confidence in your application's behavior before deploying it to users.

In this example, we will use the **GroundednessEvaluator** and **CoherenceEvaluator** from the Azure AI Evaluation SDK to assess the outputs of our existing **LangGraph**-based agent. These evaluators will help us measure how well the agent’s responses stay true to the source context and maintain logical flow throughout the conversation.


🔗 [Evaluate your Generative AI application locally with the Azure AI Evaluation SDK](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/evaluate-sdk)

In [ ]:
agent_eval = Evaluation()

## Configure tracing for Azure AI Foundry

When you build AI solutions, you want to be able to observe the behavior of your services. Observability is the ability to monitor and analyze the internal state of components within a distributed system. It is a key requirement for building enterprise-ready AI solutions.

🔗 [Inspection of telemetry data with Application Insights](https://learn.microsoft.com/en-us/semantic-kernel/concepts/enterprise-readiness/observability/telemetry-with-app-insights?tabs=Powershell&pivots=programming-language-python)

🔗 [Visualize traces on Azure AI Foundry Tracing UI](https://learn.microsoft.com/en-us/semantic-kernel/concepts/enterprise-readiness/observability/telemetry-with-azure-ai-foundry-tracing)

In [ ]:
## SEMANTICKERNEL_EXPERIMENTAL_GENAI_ENABLE_OTEL_DIAGNOSTICS_SENSITIVE=true
set_up_all(connection_string=environ.get("AZURE_INSIGHT_CONNECTION_STRING"))

In [ ]:
kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ,
    api_version="2025-01-01-preview"))



In [ ]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the tools as the source of truth; do not expose raw database internals.
- Return customer_id and product_id alongside names in all outputs.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.
"""

In [ ]:

sales_plugin = MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
)

await sales_plugin.connect()

agent = ChatCompletionAgent(
    kernel=kernel, 
    name="SalesAgent", 
    instructions=instructions,
    plugins=[sales_plugin, ]
)


In [ ]:
messages = [
    "Which is the revenue for Brake_Pads?",
    "Drill down into customer details product with the highest revenue.",
    "Which customer had the highest sales for this product?",
]


In [ ]:
async def on_intermediate_message(agent_result):

    
    # Capture assistant content
    content = agent_result.content
    if content:
        content_text = content.content if isinstance(content, ChatMessageContent) else str(content)
        await history_store.add_message(history, session_id, ChatRole.ASSISTANT, content_text)

    # Capture tool calls and results
    for item in getattr(agent_result, "items", []):
        tool_call_id = getattr(item, "call_id", None) or getattr(item, "id", None)
        if not tool_call_id:
            continue  # skip if no call_id

        # Function name for bookkeeping
        function_name = getattr(item, "function_name", "N/A")

        # Print the invocation (DEBUG ONLY — not persisted)
        if hasattr(item, "arguments"):
            print(f"Tool invocation: {function_name}({item.arguments})")

        # Extract the result content
        result_content = getattr(item, "result", item)
        if isinstance(result_content, list):
            tool_text = "\n".join([c.text if hasattr(c, "text") else str(c) for c in result_content])
        elif hasattr(result_content, "text"):
            tool_text = result_content.text
        else:
            tool_text = str(result_content)

        if function_name in tool_text:
            continue
        
        #print(f"Tool call ID: {tool_call_id}")
        #print(f"Tool Function Name: {function_name}")
        print(f"Tool output: {tool_text}")

        await history_store.add_message(
            history,
            session_id,
            ChatRole.ASSISTANT,
            content=tool_text,
            tool_call_id=tool_call_id,
            function_name=function_name
        )

       


In [ ]:
print("----- First Question -----")
print(messages[0])

await history_store.add_message(history,session_id, ChatRole.USER, messages[0])

final_response = None
async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    final_response = result 

await history_store.add_message(history,session_id, ChatRole.ASSISTANT, final_response.content.content)


In [ ]:
print(final_response)

In [ ]:


eval_results = agent_eval.evaluate(messages[0],final_response.content.content,history.messages)
print(json.dumps(eval_results, indent=4))


In [ ]:
print("----- Next Question -----")
print(messages[1])

await history_store.add_message(history,session_id, ChatRole.USER, messages[1])

final_response = None
async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    final_response = result 

await history_store.add_message(history,session_id, ChatRole.ASSISTANT, final_response.content.content)



In [ ]:
print(final_response)

In [ ]:
eval_results = agent_eval.evaluate(messages[0],final_response.content.content,history.messages)
print(json.dumps(eval_results, indent=4))

In [ ]:
print("----- Next Question -----")
print(messages[2])

await history_store.add_message(history,session_id, ChatRole.USER, messages[2])

final_response = None
async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    final_response = result 

await history_store.add_message(history,session_id, ChatRole.ASSISTANT, final_response.content.content)


In [ ]:
print(final_response)

In [ ]:
eval_results = agent_eval.evaluate(messages[2],final_response.content.content,history.messages)
print(json.dumps(eval_results, indent=4))